# Semantic Search

## The Task
The task has three parts -- data collection, data exploration / algorithm development, then finally predictive modeling.

![](http://interactive.blockdiag.com/image?compression=deflate&encoding=base64&src=eJxdjrsOwjAMRXe-wlsmRhaQkDoiMSDxBW5slahtHDmGCiH-nfQxtKy-59zruhPfUsAGPjsA56XvMdIRSIbYCZKD_RncENqQuGBQ3S7TidCwxsynjZUZ1T8m4HqvJlXZnhrBJMHBbWlTDHEeSFravYUXQy_E3TKrwbioMKb5z16UmRxfXZurVY_GjegbhqJIjaXm-wNmzE4W)

### Part I - Data Collection

Query the wikipedia API and **collect all of the articles** under the following wikipedia categories:
* [Machine Learning](https://en.wikipedia.org/wiki/Category:Machine_learning)
* [Business Software](https://en.wikipedia.org/wiki/Category:Business_software)

The code should be modular enough that any valid category from Wikipedia can be queried by the code.

The results of the query should be written to PostgreSQL tables, `page` and `category`. 
Build some sort of reference between the pages and categories. 
(Keep in mind that a page can have many categories and a category can have many pages so a straight foreign key arrangement will not work.)

In [1]:
import wikipedia
import requests
from bs4 import BeautifulSoup

In [2]:
# write a function to jsonify the category names for a category page

def jsonify_wiki_category (category_name): 
    '''
    Insert category name as a string, note that this is case sensitive
    Make sure to import requests first before using this function
    '''
    base_url = 'https://en.wikipedia.org/w/api.php'
    action = '?action=query'
    generator = '&list=categorymembers&cmtitle='
    params = '&cmlimit=max'
    form = '&format=json'
    
    category_name_url = base_url + action + generator + category_name + params + form
    category_name_response = requests.get(category_name_url)
    category_name_json = category_name_response.json()
    return category_name_json
    

In [3]:
jsonify_wiki_category('Category:Machine_learning')

{'batchcomplete': '',
 'limits': {'categorymembers': 500},
 'query': {'categorymembers': [{'ns': 0,
    'pageid': 43385931,
    'title': 'Data exploration'},
   {'ns': 0,
    'pageid': 49082762,
    'title': 'List of datasets for machine learning research'},
   {'ns': 0, 'pageid': 233488, 'title': 'Machine learning'},
   {'ns': 0, 'pageid': 53587467, 'title': 'Outline of machine learning'},
   {'ns': 0, 'pageid': 53198248, 'title': 'Singular statistical model'},
   {'ns': 0, 'pageid': 3771060, 'title': 'Accuracy paradox'},
   {'ns': 0, 'pageid': 43808044, 'title': 'Action model learning'},
   {'ns': 0,
    'pageid': 28801798,
    'title': 'Active learning (machine learning)'},
   {'ns': 0, 'pageid': 45049676, 'title': 'Adversarial machine learning'},
   {'ns': 0, 'pageid': 52642349, 'title': 'AIVA'},
   {'ns': 0, 'pageid': 30511763, 'title': 'AIXI'},
   {'ns': 0, 'pageid': 50773876, 'title': 'Algorithm Selection'},
   {'ns': 0, 'pageid': 20890511, 'title': 'Algorithmic inference'},
   

In [4]:
jsonify_wiki_category('Category:Business_software')

{'batchcomplete': '',
 'limits': {'categorymembers': 500},
 'query': {'categorymembers': [{'ns': 0,
    'pageid': 1037763,
    'title': 'Business software'},
   {'ns': 0, 'pageid': 41270069, 'title': 'AccuSystems'},
   {'ns': 0, 'pageid': 5211212, 'title': 'Active policy management'},
   {'ns': 0, 'pageid': 28502793, 'title': 'Alexandria (library software)'},
   {'ns': 0, 'pageid': 44133735, 'title': 'Alteryx'},
   {'ns': 0, 'pageid': 12715119, 'title': 'Amadeus CRS'},
   {'ns': 0, 'pageid': 24061342, 'title': 'AMS Device Manager'},
   {'ns': 0, 'pageid': 54594603, 'title': 'Angelfish software'},
   {'ns': 0, 'pageid': 1762176, 'title': 'Applicant tracking system'},
   {'ns': 0, 'pageid': 22847264, 'title': 'Application retirement'},
   {'ns': 0,
    'pageid': 35959361,
    'title': 'Architecture of Interoperable Information Systems'},
   {'ns': 0, 'pageid': 19657756, 'title': 'Asset recovery software'},
   {'ns': 0, 'pageid': 53113973, 'title': 'Avaloq'},
   {'ns': 0, 'pageid': 340265

In [69]:
title_stuff = (jsonify_wiki_category('Category:Machine_learning')['query']['categorymembers'])
type(title_stuff)

list

In [73]:
title_stuff[:5]

[{'ns': 0, 'pageid': 43385931, 'title': 'Data exploration'},
 {'ns': 0,
  'pageid': 49082762,
  'title': 'List of datasets for machine learning research'},
 {'ns': 0, 'pageid': 233488, 'title': 'Machine learning'},
 {'ns': 0, 'pageid': 53587467, 'title': 'Outline of machine learning'},
 {'ns': 0, 'pageid': 53198248, 'title': 'Singular statistical model'}]

In [131]:
title_stuff[1]['title']

'List of datasets for machine learning research'

In [115]:
[list(dicts.items()) for dicts in title_stuff][0]

[('title', 'Data exploration'), ('ns', 0), ('pageid', 43385931)]

In [128]:
def get_article_titles (category_name):
    title_dicts = (jsonify_wiki_category(category_name)['query']['categorymembers'])
    title_tuples = [list(dicts.items() for dicts in title_dicts)]
    print(title_tuples[0][0][1])

In [129]:
get_article_titles('Category:Machine_learning')

TypeError: 'dict_items' object does not support indexing

In [25]:
len(jsonify_wiki_category('Category:Machine_learning')['query']['categorymembers'])

228

In [6]:
jsonify_wiki_category('Category:Machine_learning')['query']['categorymembers'][0]['title']

'Data exploration'

In [ ]:
# write a function that jsonifies the articles

def jsonify_wiki_article (article_name): 
    '''
    Insert category name as a string, note that this is case sensitive
    Make sure to import requests first before using this function
    '''
    base_url = 'https://en.wikipedia.org/w/api.php'
    action = '?action=parse'
    prop = '&page='
    form = '&format=json'
    
    article_name_url = base_url + action + prop + article_name + form
    article_name_response = requests.get(article_name_url)
    article_name_json = article_name_response.json()
    return article_name_json
    

In [ ]:
# get the article in html format

def htmlify_wiki_article (article_name_json):
    article_name_html = article_name['parse']['text']['*']
    return article_name_html

In [ ]:
# run beautiful soup on the html format to do a 1st pass at cleaning the html to just text

def beautify_html_article (article_name_html):
    soup = BeautifulSoup(article_name_html, 'html.parser')
    article_text = soup.get_text().replace('\n', '')
    return article_text

In [ ]:

beautify_html_article(htmlify_wiki_article(jsonify_wiki_article(jsonify_wiki_category('Category:Business_software')
                                                                ['query']['categorymembers'][0]['title'])))
